In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import math

In [2]:
dist = pd.read_csv('dist_data.csv', sep=';')

In [3]:
x = dist.iloc[:, :-1].values
y = dist.iloc[:, 1].values

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [5]:
def clean_sample(sample):
    sample_list = sample[0].replace(', [', '; [').split(';')
    sample_x = sample_list[0]
    sample_Px = sample_list[1]

    sample_x_list = []
    sample_Px_list = []
    
    for x in sample_x.split(','):
        sample_x_list.append(x.replace(' ', '').replace('[', '').replace(']', ''))
    for Px in sample_Px.split(','):
        sample_Px_list.append(Px.replace(' ', '').replace('[', '').replace(']', ''))

    dict_sample = {}
    for i in range(len(sample_x_list)):
        dict_sample[sample_x_list[i]] = sample_Px_list[i]
    return dict_sample

In [6]:
def all_keys_dataset1_exist_in_dataset2(dataset1, dataset2, threshold):
    count = 0
    total = len(dataset1.keys())
    for key in dataset1.keys():
        if dataset2.get(key) is None:
            pass
        else:
            count += 1

    if (count/total) * 100 >= float(threshold):
        return True
    else:
        return False

In [87]:
def calculate_distance(distance_func, min_precent_similarity):
    count = 0
    result = []
    for i, sample_x_test in enumerate(x_test): 
        result_zikma = []
    
        for j, sample_x_train in enumerate(x_train):
            if all_keys_dataset1_exist_in_dataset2(clean_sample(sample_x_test), clean_sample(sample_x_train), min_precent_similarity):
    
                Px_list = []
                for xd1, Pxd1 in clean_sample(sample_x_test).items():
                    if xd1 == "" or Pxd1 == "":
                        continue
                    Pxd2 = clean_sample(sample_x_train).get(xd1)
                    if Pxd2 is None:
                        continue
                    Px_list.append([Pxd1, Pxd2])
            
                sum = distance_func(Px_list)
                result_zikma.append([y_train[j], sum])
                
            else:
                continue
        
        result.append([sample_x_test ,result_zikma])
        
        count += 1
        if count > 100:
            break
    
    return result

In [88]:
def JS_Div(p, q):
    p = np.asarray(p)
    q = np.asarray(q)
    # normalize
    p /= p.sum()
    q /= q.sum()
    m = (p + q) / 2
    return (KL_div(p, m) + KL_div(q, m)) / 2

In [89]:
def kl(Px_list):
    sum = 0 
    for i in Px_list:
        Pxd1 = i[0]
        Pxd2 = i[1]
        sum += float(Pxd1) * math.log10(float(Pxd1) / float(Pxd2))
    return sum

In [ ]:
result = []
for i in range(1, 100): # one or zero
    result.append(calculate_distance(kl, i))
    if i == 2:
        break

In [65]:
#result = calculate_distance(kl, 70)

In [85]:
#using max
y_pred = []
for i in range(len(x_test) - 2010):
    t = pd.DataFrame(result[0][i][1], columns=['Label', 'Kl Result'])
    bests = t.value_counts()
    if bests.size == 0:
        y_pred.append(0)
    else:
        y_pred.append(bests.head(1).index[0][0])

In [86]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
res = confusion_matrix(y_test[:101], y_pred)
print("Confusion Matrix:")
print(res)
res1 = classification_report(y_test[:101], y_pred)
print("Classification Report:",)
print (res1)
res2 = accuracy_score(y_test[:101],y_pred)
print("Accuracy:",res2)

Confusion Matrix:
[[ 6  0  0  1  0  0  0]
 [ 0 21  0  5  0  0  0]
 [ 0  0  1  0  0  0  0]
 [ 0  2  0 42  0  0  0]
 [ 0  1  1 12  3  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  4  0  0  0  0  0]]
Classification Report:
              precision    recall  f1-score   support

       27311       1.00      0.86      0.92         7
       27321       0.75      0.81      0.78        26
       27341       0.50      1.00      0.67         1
       27523       0.68      0.95      0.79        44
       27525       1.00      0.18      0.30        17
       27652       0.00      0.00      0.00         2
       28090       0.00      0.00      0.00         4

    accuracy                           0.72       101
   macro avg       0.56      0.54      0.49       101
weighted avg       0.73      0.72      0.67       101

Accuracy: 0.7227722772277227


/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,